In [1]:
!pip install simple-ddl-parser

  Using cached simple_ddl_parser-1.7.1-py3-none-any.whl.metadata (40 kB)
Using cached simple_ddl_parser-1.7.1-py3-none-any.whl (83 kB)


In [1]:
from simple_ddl_parser import DDLParser

In [2]:
ddl_1 = """create table dev.data_sync_history(
    data_sync_id bigint not null,
    sync_count bigint not null,
    sync_mark timestamp  not  null,
    sync_start timestamp  not null,
    sync_end timestamp  not null,
    message varchar(2000) null,
    primary key (data_sync_id, sync_start)
);
CREATE TABLE orders (
    order_id INT IDENTITY(1,1) PRIMARY KEY COMMENT 'Unique identifier for an order',
    customer_id INT NOT NULL COMMENT 'Foreign key referencing customers table',
    order_date DATE NOT NULL COMMENT 'Date the order was placed',
    total_amount DECIMAL(10,2) NOT NULL COMMENT 'Total amount of the order' default 0,
    CONSTRAINT fk_orders_customers FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
"""

In [3]:
parse_results = DDLParser(ddl_1).run()

In [4]:
parse_results

[{'table_name': 'data_sync_history',
  'schema': 'dev',
  'primary_key': ['data_sync_id', 'sync_start'],
  'columns': [{'name': 'data_sync_id',
    'type': 'bigint',
    'size': None,
    'references': None,
    'unique': False,
    'nullable': False,
    'default': None,
    'check': None},
   {'name': 'sync_count',
    'type': 'bigint',
    'size': None,
    'references': None,
    'unique': False,
    'nullable': False,
    'default': None,
    'check': None},
   {'name': 'sync_mark',
    'type': 'timestamp',
    'size': None,
    'references': None,
    'unique': False,
    'nullable': False,
    'default': None,
    'check': None},
   {'name': 'sync_start',
    'type': 'timestamp',
    'size': None,
    'references': None,
    'unique': False,
    'nullable': False,
    'default': None,
    'check': None},
   {'name': 'sync_end',
    'type': 'timestamp',
    'size': None,
    'references': None,
    'unique': False,
    'nullable': False,
    'default': None,
    'check': None},
 

In [7]:
!pip install ddlparse openpyxl

  Using cached ddlparse-1.10.0-py3-none-any.whl.metadata (6.5 kB)
Using cached ddlparse-1.10.0-py3-none-any.whl (10 kB)


In [8]:
!pip install numpy

In [9]:
!pip install pandas

In [7]:
import pandas as pd

In [14]:
import re

def generate_dataframe_from_sql(sql_file_path):
    try:
        # Step 1: Read SQL content
        with open(sql_file_path, 'r') as file:
            ddl_content = file.read()

        # Step 2: Parse SQL using DDLParser
        parser = DDLParser(ddl_content)
        parsed_data = parser.run()  # Returns a list of table definitions

        if not parsed_data:
            raise ValueError("Parsing failed. Ensure SQL syntax is correct.")

        # Step 3: Create an empty pandas DataFrame with the required columns
        columns = [ "TableName", "Columns", "DataFamily", "MinValue", "MaxValue", "CustomDirectListValue", "fk_TableName", "fk_ColumnName", "Description"]
        df = pd.DataFrame(columns=columns)

        # Step 4: Populate the DataFrame with parsed data
        for table_info in parsed_data:
            table_name = table_info["table_name"]
            fk_constraints = table_info.get("constraints", {}).get("references", [])

            for column in table_info["columns"]:
                # Default foreign key info to None
                fk_table = None
                fk_column = None

                # Check if the column is part of any foreign key constraint
                for fk in fk_constraints:
                    if column["name"] in fk["columns"]:
                        fk_table = fk["table"]
                        fk_column = fk["columns"][0]  # Assuming single-column foreign keys
                        break

                # Get the description/comment
                description = column.get("comment", "")
                data_family = "Unknown"  # Default or derived value
                min_value = None  # Populate based on column type or constraints
                max_value = None  # Populate based on column type or constraints
                custom_direct_list_value = None  # Populate based on specific rules

                # Add a row to the DataFrame
                df = pd.concat([df, pd.DataFrame([{
                "TableName": table_name,
                "Columns": column["name"],
                "DataFamily": data_family,
                "MinValue": min_value,
                "MaxValue": max_value,
                "CustomDirectListValue": custom_direct_list_value,
                "fk_TableName": fk_table,
                "fk_ColumnName": fk_column,
                "Description": description
                }])], ignore_index=True)

        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
# df = generate_dataframe_from_sql("input.sql")
# print(df)


In [15]:
generate_dataframe_from_sql("CREATE_TABLE_dev.sql")

,TableName,Columns,DataFamily,MinValue,MaxValue,CustomDirectListValue,fk_TableName,fk_ColumnName,Description
0,data_sync_history,data_sync_id,Unknown,None,None,None,None,None,
1,data_sync_history,sync_count,Unknown,None,None,None,None,None,
2,data_sync_history,sync_mark,Unknown,None,None,None,None,None,
3,data_sync_history,sync_start,Unknown,None,None,None,None,None,
4,data_sync_history,sync_end,Unknown,None,None,None,None,None,
5,data_sync_history,message,Unknown,None,None,None,None,None,
6,orders,order_id,Unknown,None,None,None,None,None,'Unique identifier for an order'
7,orders,customer_id,Unknown,None,None,None,customers,customer_id,'Foreign key referencing customers table'
8,orders,order_date,Unknown,None,None,None,None,None,'Date the order was placed'
9,orders,total_amount,Unknown,None,None,None,None,None,'Total amount of the order'


In [ ]:
async def process_sql_file(uploaded_file):
    """
    Accept an uploaded .xlsx, .sql, or .ddl file, process it, filter by distinct table names, 
    and return each as a JSON response.
    """
    
    # Process the .sql or .ddl file
    # Step 1: Read the file content
    with open(uploaded_file, 'r') as file:
            contents = await file.read()
    
    ddl_content = contents.decode('utf-8')  # Decode bytes to string
    logger.info("sql file is loaded")
    # Step 2: Parse the SQL using DDLParser
    parser = DDLParser(ddl_content)
    parsed_data = parser.run()
    logger.info("sql data parsed")

    # Step 3: Create a pandas DataFrame with the required structure
    columns = ["TableName", "Columns", "fk_TableName", "fk_ColumnName", "Description"]
    df = pd.DataFrame(columns=columns)

    # Step 4: Populate the DataFrame
    for table_info in parsed_data:
        table_name = table_info["table_name"]
        fk_constraints = table_info.get("constraints", {}).get("references", [])

        for column in table_info["columns"]:
            # Default foreign key info to None
            fk_table = None
            fk_column = None

            # Check if the column is part of any foreign key constraint
            for fk in fk_constraints:
                if column["name"] in fk["columns"]:
                    fk_table = fk["table"]
                    fk_column = fk["columns"][0]  # Assuming single-column foreign keys
                    break

            # Get the description/comment
            description = column.get("comment", "")

            # Add a row to the DataFrame
            df = pd.concat([df, pd.DataFrame([{
                "TableName": table_name,
                "Columns": column["name"],
                "fk_TableName": fk_table,
                "fk_ColumnName": fk_column,
                "Description": description
            }])], ignore_index=True)

    return df


In [12]:
import io
import pandas as pd
from fastapi import HTTPException

async def process_file(uploaded_file):
    """
    Accept an uploaded .xlsx, .sql, or .ddl file, process it, filter by distinct table names, 
    and return each as a JSON response.
    """
    if uploaded_file.filename.endswith('.xlsx'):
        # Load the uploaded .xlsx file into a DataFrame
        contents = await uploaded_file.read()
        df = pd.read_excel(io.BytesIO(contents), sheet_name="ColumnInfo", keep_default_na=False)

        # Check if 'TableName' exists in the columns
        if 'TableName' not in df.columns:
            raise HTTPException(detail="The 'TableName' column was not found in the template.")
    elif uploaded_file.filename.endswith(('.sql', '.ddl')):
        # Process the .sql or .ddl file
        # Step 1: Read the file content
        contents = await uploaded_file.read()
        ddl_content = contents.decode('utf-8')  # Decode bytes to string

        # Step 2: Parse the SQL using DDLParser
        parser = DDLParser(ddl_content)
        parsed_data = parser.run()

        # Step 3: Create a pandas DataFrame with the required structure
        columns = ["TableName", "Columns", "fk_TableName", "fk_ColumnName", "Description"]
        df = pd.DataFrame(columns=columns)

        # Step 4: Populate the DataFrame
        for table_info in parsed_data:
            table_name = table_info["table_name"]
            fk_constraints = table_info.get("constraints", {}).get("references", [])

            for column in table_info["columns"]:
                # Default foreign key info to None
                fk_table = None
                fk_column = None

                # Check if the column is part of any foreign key constraint
                for fk in fk_constraints:
                    if column["name"] in fk["columns"]:
                        fk_table = fk["table"]
                        fk_column = fk["columns"][0]  # Assuming single-column foreign keys
                        break

                # Get the description/comment
                description = column.get("comment", "")

                # Add a row to the DataFrame
                df = pd.concat([df, pd.DataFrame([{
                    "TableName": table_name,
                    "Columns": column["name"],
                    "fk_TableName": fk_table,
                    "fk_ColumnName": fk_column,
                    "Description": description
                }])], ignore_index=True)
    else:
        # Raise an error for unsupported file types
        raise HTTPException(detail="Invalid file type. Please upload an .xlsx, .sql, or .ddl file.")

    # Extract distinct table names
    unique_table_names = df['TableName'].unique()

    # Initialize the dictionary to hold filtered data for each table
    template_data = {}

    # Filter records by each table name and store in dictionary
    for table_name in unique_table_names:
        # Filter DataFrame for the current table name
        filtered_df = df[df['TableName'] == table_name]

        # Convert the filtered DataFrame to a list of dictionaries
        template_data[table_name] = filtered_df.to_dict(orient="records")

    # Return the JSON response
    return template_data


In [15]:
process_file("CREATE TABLE dev.sql")

<coroutine object process_file at 0x0000023ABEC2DD40>

In [12]:
import openpyxl
from openpyxl import Workbook
import re

def generate_template_from_sql(sql_file_path, output_excel_path):
    # Step 1: Read SQL content
    with open(sql_file_path, 'r') as file:
        ddl_content = file.read()

    # Step 2: Extract all table names using a regular expression
    table_names = re.findall(r'CREATE TABLE\s+`?(\w+(\.\w+)?)`?\s*\(', ddl_content, re.IGNORECASE)
    table_names = [name[0] for name in table_names]  # Flatten the list to just table names

    # Step 3: Parse SQL using DDLParser
    parser = DDLParser(ddl_content)
    parsed_data = parser.run()  # This returns a list, not a dict

    # Step 4: Create a new Excel file and add "ColumnInfo" sheet
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "ColumnInfo"

    # Step 5: Add headers to the "ColumnInfo" sheet
    headers = ["TableName", "Columns", "fk_TableName", "fk_ColumnName", "Description"]
    sheet.append(headers)

    # Step 6: Populate the "ColumnInfo" sheet with parsed data
    table_index = 0  # To keep track of which table we're currently processing
    for table_info in parsed_data:
        # Get the correct table name from the list
        table_name = table_names[table_index]
        fk_constraints = table_info.get("constraints", {}).get("references", [])

        for column in table_info["columns"]:
            # Default foreign key info to None
            fk_table = None
            fk_column = None

            # Check if the column is part of any foreign key constraint
            for fk in fk_constraints:
                if column["name"] in fk["columns"]:
                    fk_table = fk["table"]
                    fk_column = fk["columns"][0]  # Assuming single-column foreign keys for now
                    break

            # Get the description/comment
            description = column.get("comment", "")

            # Append row data with the correct table name
            sheet.append([table_name, column["name"], fk_table, fk_column, description])

        # Move to the next table name in the list
        table_index += 1

    # Step 7: Save the new Excel file
    workbook.save(output_excel_path)
    print(f"Excel file created: {output_excel_path}")


In [13]:
generate_template_from_sql("create_new_table.txt",  "New_Output1.xlsx")

Excel file created: New_Output1.xlsx
